# Places of origin by last name

In [ ]:
# Get the name of the database file and make it read-only
from pathlib import Path

# Note: this assumes you have copied the `ncvoters.db` database to your home directory
dbfile = Path.home().joinpath("ncvoters.db")
print(f"Database file is {dbfile}")

Query to be used:

In [ ]:
query_sql = """
    SELECT		distinct last_name, birth_state, sum(one) as nbr
    FROM (
        SELECT		last_name, birth_state, 1 as one
        FROM		voters
        WHERE		last_name is not NULL
        AND			trim(last_name) != ""
        AND			birth_state is not NULL
        AND			trim(birth_state) != ""
    )
    GROUP BY		last_name, birth_state
    HAVING			nbr > 10
    ORDER BY		1, 3 DESC
;
"""

Run the query and read the results

In [ ]:
import sqlite3

with sqlite3.connect(dbfile) as con:
    c = con.cursor()
    res = c.execute(query_sql)
    prev_last_name = None
    state_list = []
    for row in res.fetchall():
        last_name, birth_state, count = row
        if last_name != prev_last_name:
            if prev_last_name is not None:
                print(f"{prev_last_name:16s} {state_list}")
            state_list.clear()
        if len(state_list) < 5:
            state_list.append(birth_state)
        prev_last_name = last_name